In [ ]:
import csv
import random
import heapq


def generate_graph(csv_file):
    graph = {}

    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)

        for row in reader:
            source = row['full_id']
            target = row['full_id_2']

            weight = random.randint(1, 100)

            if source not in graph:
                graph[source] = []

            if target not in graph:
                graph[target] = []

            graph[source].append((target, weight))
            graph[target].append((source, weight))

    return graph


class DisjointSetVariant:
    def __init__(self, n):
        self.parent = list(range(n + 1))
        self.rank = [0] * (n + 1)

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x != root_y:
            if self.rank[root_x] < self.rank[root_y]:
                self.parent[root_x] = root_y
            elif self.rank[root_x] > self.rank[root_y]:
                self.parent[root_y] = root_x
            else:
                self.parent[root_y] = root_x
                self.rank[root_x] += 1


def kruskal_max_spanning_tree(edges, num_vertices):
    edges.sort(key=lambda x: x[2], reverse=True)

    ds = DisjointSetVariant(num_vertices)

    max_spanning_tree = []

    for u, v, weight in edges:
        if ds.find(u) != ds.find(v):
            ds.union(u, v)
            max_spanning_tree.append((u, v, weight))

    return max_spanning_tree


def prim_max_spanning_tree(graph):
  # Invertir los pesos de las aristas
  inverted_graph = [(u, v, -weight) for u, v, weight in graph]

  # Crear un diccionario para almacenar las aristas del árbol de expansión máxima
  max_spanning_tree = {}

  # Elegir un nodo inicial de forma aleatoria
  start = inverted_graph[0][0]

  # Crear un conjunto para almacenar los nodos visitados
  visited = set([start])

  # Crear una lista de aristas candidatas para ser consideradas en cada iteración
  candidates = []
  for edge in inverted_graph:
    if edge[0] == start:
      heapq.heappush(candidates, edge)

  # Ejecutar el algoritmo hasta que no haya más candidatos
  while candidates:
    # Obtener la arista con el mayor peso
    u, v, weight = heapq.heappop(candidates)

    # Si el nodo 'v' no ha sido visitado, agregar la arista al árbol de expansión máxima
    if v not in visited:
      visited.add(v)
      max_spanning_tree[(u, v)] = -weight  # Invertir el peso nuevamente

      # Agregar las aristas adyacentes del nodo 'v' a los candidatos
      for edge in inverted_graph:
        if edge[0] == v and edge[1] not in visited:
          heapq.heappush(candidates, edge)

  return list(max_spanning_tree.keys())

mst_prim = prim_max_spanning_tree(graph)


def dijkstra_shortest_path(graph, start, end):
    distances = {vertex: float('inf') for vertex in graph}
    distances[start] = 0

    previous = {vertex: None for vertex in graph}

    min_heap = [(0, start)]

    while min_heap:
        current_distance, current_vertex = heapq.heappop(min_heap)

        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex]:
            distance = current_distance + weight

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous[neighbor] = current_vertex
                heapq.heappush(min_heap, (distance, neighbor))

    if distances[end] == float('inf'):
        return None

    path = []
    current = end
    while current is not None:
        path.append(current)
        current = previous[current]

    path.reverse()
    return path


def show_route(route):
    print("Ruta encontrada:")
    print(" -> ".join(str(node) for node in route))

def show_route_prim(mst_prim ):
  ids = set()
  for u, v in mst_prim :
    ids.add(u)
    ids.add(v)

  sorted_ids = sorted(ids)

  print(" -> ".join(f"{node_id}" for node_id in sorted_ids))

def main():
    file_name = 'lavictoria-streets-intersections.csv'
    graph = generate_graph(file_name)

    edge_list = []
    vertex_index = {}
    index = 1

    for source, adjacents in graph.items():
        if source not in vertex_index:
            vertex_index[source] = index
            index += 1

        for target, weight in adjacents:
            if target not in vertex_index:
                vertex_index[target] = index
                index += 1

            edge_list.append((vertex_index[source], vertex_index[target], weight))

    num_vertices = len(vertex_index)

    

    algorithm = input("Seleccione el algoritmo a utilizar (kruskal / prim / dijkstra): ")

    if algorithm == "kruskal":
        max_kruskal = kruskal_max_spanning_tree(edge_list, num_vertices)      
        show_route(max_kruskal)
    elif algorithm == "prim":
        show_route_prim(mst_prim)
    elif algorithm == "dijkstra":
        start_street = input("Ingrese la calle de origen: ")
        end_street = input("Ingrese la calle de destino: ")
        shortest_path = dijkstra_shortest_path(graph, start_street, end_street)
        show_route(shortest_path)
    else:
        print("Algoritmo no válido. Por favor, seleccione 'kruskal', 'prim' o 'dijkstra'.")



if __name__ == "__main__":
    main()